<a href="https://colab.research.google.com/github/jaisal1311/Deep-Learning/blob/master/Computer%20Vision/Wine%20Quality%20Predictor/Wine_Reggression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install jovian

     |████████████████████████████████| 92kB 3.5MB/s 
  Created wheel for uuid: filename=uuid-1.30-cp36-none-any.whl size=6502 sha256=c75d9f6c321595c0a94aab0b166731784cff6ea6d74195a59b094a148d643b8d
  Stored in directory: /root/.cache/pip/wheels/2a/80/9b/015026567c29fdffe31d91edbe7ba1b17728db79194fca1f21
Successfully built uuid


In [ ]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.functional as F
import torchvision.transforms as tranform
import jovian

In [ ]:
df = pd.read_csv('wine.csv')

In [ ]:
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [ ]:
df.isnull().any()

fixed acidity           False
volatile acidity        False
citric acid             False
residual sugar          False
chlorides               False
free sulfur dioxide     False
total sulfur dioxide    False
density                 False
pH                      False
sulphates               False
alcohol                 False
quality                 False
dtype: bool

In [ ]:
input_cols = list(df.columns)[:-1]
input_cols

['fixed acidity',
 'volatile acidity',
 'citric acid',
 'residual sugar',
 'chlorides',
 'free sulfur dioxide',
 'total sulfur dioxide',
 'density',
 'pH',
 'sulphates',
 'alcohol']

In [ ]:
output_cols = ['quality']

In [ ]:
df1 = df.copy(deep = True)
input_array = df1[input_cols].to_numpy()
output_array = df1[output_cols].to_numpy()

In [ ]:
print(input_array)
print(output_array)

[[ 7.4    0.7    0.    ...  3.51   0.56   9.4  ]
 [ 7.8    0.88   0.    ...  3.2    0.68   9.8  ]
 [ 7.8    0.76   0.04  ...  3.26   0.65   9.8  ]
 ...
 [ 6.3    0.51   0.13  ...  3.42   0.75  11.   ]
 [ 5.9    0.645  0.12  ...  3.57   0.71  10.2  ]
 [ 6.     0.31   0.47  ...  3.39   0.66  11.   ]]
[[5]
 [5]
 [5]
 ...
 [6]
 [5]
 [6]]


In [ ]:
inputs = torch.Tensor(input_array)
targets = torch.Tensor(output_array)
print(inputs)

print(targets)

tensor([[ 7.4000,  0.7000,  0.0000,  ...,  3.5100,  0.5600,  9.4000],
        [ 7.8000,  0.8800,  0.0000,  ...,  3.2000,  0.6800,  9.8000],
        [ 7.8000,  0.7600,  0.0400,  ...,  3.2600,  0.6500,  9.8000],
        ...,
        [ 6.3000,  0.5100,  0.1300,  ...,  3.4200,  0.7500, 11.0000],
        [ 5.9000,  0.6450,  0.1200,  ...,  3.5700,  0.7100, 10.2000],
        [ 6.0000,  0.3100,  0.4700,  ...,  3.3900,  0.6600, 11.0000]])
tensor([[5.],
        [5.],
        [5.],
        ...,
        [6.],
        [5.],
        [6.]])


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, random_split
dataset = TensorDataset(inputs, targets)

In [ ]:
num_obs = len(df)
test_size = int(num_obs * 0.3)
train_size = num_obs - test_size
train_ds, test_ds = random_split(dataset, [train_size, test_size])
train_ds

In [ ]:
batch_size = 32
train_loader = DataLoader(train_ds, batch_size, shuffle = True)
test_loader = DataLoader(test_ds, batch_size)

In [ ]:
for x, y in train_loader:
  print(len(x), len(y))
  break

32 32


In [ ]:
input_dim = len(input_cols)
output_dim = len(output_cols)

In [ ]:
class Wine(nn.Module):

  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(input_dim, output_dim)
    
  def forward(self, xb):
    out = self.linear(xb)
    return out

  def training_step(self, batch):
    inputs, targets = batch
    out = self(inputs) 
    loss = torch.nn.functional.l1_loss(out, targets)
    return loss

  def testing_step(self, batch):
    inputs, targets = batch
    out = self(inputs) 
    loss = torch.nn.functional.l1_loss(out, targets)
    return {'value_loss': loss.detach()}

  def testing_epoch_end(self, outputs):
    batch_losses = [x['value_loss'] for x in outputs]
    epoch_loss = torch.stack(batch_losses).mean()
    return epoch_loss

  def epoch_end(self, epoch, result, num_epochs):
    if((epoch + 1) % 20 == 0 or epoch == num_epochs - 1):
      print('epoch: {[]} , loss: {:.4f} '.format(epoch + 1, result['value_loss']) )
    

model = Wine()


In [ ]:
def evaluate(model, test_loader):
  outputs = [model.testing_step(batch) for batch in test_loader]
  return model.testing_epoch_end(outputs)

In [ ]:
epoch = 900
lr = 0.001


In [ ]:
def fit(epochs, lr, model,train_loader, test_loader, opt_func = torch.optim.SGD):
  history = []
  opt = opt_func(model.parameters(), lr)
  for epoch in range(epochs):
    for batch in train_loader:
      loss = model.training_step(batch)
      loss.backward()
      opt.step()
      opt.zero_grad()
    result = evaluate(model, test_loader)
    # model.epoch_end(epoch, result, epochs)
    history.append(result)
  return history

In [ ]:
history = fit(epoch, lr, model, train_loader, test_loader)

In [ ]:
history = fit(epoch, 0.01, model, train_loader, test_loader)

In [ ]:
history = fit(epoch, 0.001, model, train_loader, test_loader)

In [ ]:
exp_loss = evaluate(model, test_loader)
print(exp_loss)

tensor(1.2915)


In [ ]:
inp, out = test_ds[0]
print(inp)
inp = inp.unsqueeze(0)
pred = model(inp)
print(out)
print(int(pred[0].detach()))

tensor([ 9.5000,  0.4600,  0.4900,  6.3000,  0.0640,  5.0000, 17.0000,  0.9988,
         3.2100,  0.7300, 11.0000])
tensor([6.])
6
